# Demonstration of low-level SQL & PreCalc functions within ChebyChecker
 - This is likely to mainly be of use while developing code 
 - MJP

In [3]:
import time
import numpy as np
import scipy.stats as stats
import math
import random
from collections import defaultdict
import os
import sys
from collections import Counter
import glob 
import warnings
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import importlib
from astropy.time import Time
import pickle


# -------------------------------------------------------------------------------------
# Local imports
# -------------------------------------------------------------------------------------

# parent directory is */cheby_checker
HEAD_DIR = os.path.dirname(os.path.realpath(os.getcwd())) 
sys.path.append(os.path.join(HEAD_DIR))
print(f' HEAD_DIR: {HEAD_DIR} ')

from cheby_checker import sql
from cheby_checker import orbit_cheby
from cheby_checker import nbody_reader
from cheby_checker import obs_pos
from cheby_checker import sql
from cheby_checker import precalc



 HEAD_DIR: /Users/matthewjohnpayne/Envs/cheby_checker 


# Notes on Direct Access to SQL
 - Many of the functions below read/write to/from a SQLite database
 - It's likely to be of interest to some to go and look in the database by hand (on the command line)
 - Here are some basic commands to get you going

#### Location of sqldb is going to be something like
 - ~/.cheby_checker_data/cheby_checker.db 

#### Log into the database
 - sqlite3 ~/.cheby_checker_data/cheby_checker.db 
    
#### See the schema for all tables in the db
 - .schema

#### See the schema for a specific table
 - .schema object_desig

#### Examine the contents of a specific table 
 - select * FROM object_coefficients ;

#### More knowledge
 - https://sqlite.org/cli.html

# Background
 - I expect that the "orbit_cheby" module exists and has functionality to take the results of an n-body integration and convert them into a dictionary of chebyshev coefficients (with seprate coefficient sets for each "sector" of time)  - There is a single dictionary for each object


# --------- Demo-Data Functions : Create MSC ---------------------

 - These functions create fake data for the sake of subsequently demonstrating PreCalc / SQL functionalities

In [4]:
def create_sample_cheby_dict(n=1):
    assert n < 50 , 'Dont make that many objects ...!'
    ''' convenience function to make demo MSC-list '''
    importlib.reload(nbody_reader)
    importlib.reload(orbit_cheby)

    # Pre-populate numpy array by reading from demo file ...
    text_filepath = os.path.join(os.path.dirname(os.getcwd() ), 'dev_data', '2022AA_demo.txt')
    assert os.path.isfile(text_filepath)
    name, times, states  = nbody_reader.parse_nbody_txt( text_filepath )
    
    print(f'max(times):{np.max(times)}')
    
    if n ==1 :
        # Create MSCs & return 
        return orbit_cheby.MSC_Loader(FROM_ARRAY = True , 
                                primary_unpacked_provisional_designations = name, 
                                times_TDB = times, 
                                statearray = states).MSCs        
    else:
        # Pre-populate numpy array ...
        # - Note that we need to allow for MULTIPLE NAMES
        # - We are creating a 3D array of input coords
        arrays, names = [],[] 
        for i in range(0,n):
            # Varying the names 
            names.append(str(int(name[:4]) + i ) + name[4:])
            # Varying the properties of the states 
            rand = 1 + np.random.random()/10
            arrays.append(states*rand)
        # Stack-up the multiple arrays 
        states_3D = np.stack(arrays, axis=2)
        states_3D.shape

        # Create MSCs & return 
        return orbit_cheby.MSC_Loader(FROM_ARRAY = True , 
                                primary_unpacked_provisional_designations = names, 
                                times_TDB = times, 
                                statearray = states_3D).MSCs

In [5]:
n=3 
MSCs = create_sample_cheby_dict( n=n )

print( 'Returned variable is of type %r , and length %r  ' % (type(MSCs),len(MSCs) ) )

for i in range(n):
    M = MSCs[i]
    print( f'i:{i}, type', type(M) )
    print ( 'Number of sectors = ' , len(M.sector_coeffs) )
    print ( 'Shape of sector[0] = ' , M.sector_coeffs[0].shape )
    print ( 'Shape of sector[1] = ' , M.sector_coeffs[1].shape )
    print ( 'Shape of sector[517] = ' , M.sector_coeffs[517].shape )
    print ( 'Range of validity : ', M.get_valid_range_of_dates() ) 
    print ( 'primary_unpacked_provisional_designation : ', M.primary_unpacked_provisional_designation )
    print()
    del M 


max(times):2459999.0
INIT MSC_Loader...
Returned variable is of type <class 'list'> , and length 3  
i:0, type <class 'cheby_checker.orbit_cheby.MSC'>
Number of sectors =  625
Shape of sector[0] =  (8, 27)
Shape of sector[1] =  (8, 27)
Shape of sector[517] =  (8, 27)
Range of validity :  (2440000, 2459999.999999)
primary_unpacked_provisional_designation :  2022 AA

i:1, type <class 'cheby_checker.orbit_cheby.MSC'>
Number of sectors =  625
Shape of sector[0] =  (8, 27)
Shape of sector[1] =  (8, 27)
Shape of sector[517] =  (8, 27)
Range of validity :  (2440000, 2459999.999999)
primary_unpacked_provisional_designation :  2023 AA

i:2, type <class 'cheby_checker.orbit_cheby.MSC'>
Number of sectors =  625
Shape of sector[0] =  (8, 27)
Shape of sector[1] =  (8, 27)
Shape of sector[517] =  (8, 27)
Range of validity :  (2440000, 2459999.999999)
primary_unpacked_provisional_designation :  2024 AA



# --------- Demo-Data Functions : Create Observatory Positions ----------

- Create a sample of observatory-positions for the sake of subsequently demonstrating PreCalc / SQL functionalities


# *** !!! THIS TAKES A LONG TIME TO RUN !!! ***

 - IT'S *NOT* PART OF orbit_cheby BUT WE'LL STILL NEED TO USE A FASTER VERSION 

In [6]:
%%time
importlib.reload(obs_pos)
#
# *** *** *** *** *** *** *** THIS TAKES ~200 SECONDS TO RUN *** *** *** *** *** *** ***
#
# Create a sample of observatory-positions as inputs 
# - Note that the NBody code uses HELIOCENTRIC EQUATORIAL coords
# - So the Cheby's need to work-in/evaluate-to HELIOCENTRIC EQUATORIAL coords
# - Hence the observatory-positions will need to be in HELIOCENTRIC EQUATORIAL coords
# - We will use the ObsPos class to get this data (see '' demo notebook for more details)
JDlist = orbit_cheby.Base().JDlist 
observatoryXYZ = np.array( [obs_pos.ObsPos().get_heliocentric_equatorial_xyz(jd , obsCode='500') for jd in JDlist] ).T
print(len(JDlist) , JDlist[:3], '...',JDlist[-3:])
print(len(observatoryXYZ) , observatoryXYZ[:3], '...',observatoryXYZ[-3:])


20000 [2440000 2440001 2440002] ... [2459997 2459998 2459999]
3 [[-0.46118508 -0.44608068 -0.43084806 ... -0.87534198 -0.88349422
  -0.89137455]
 [-0.82722792 -0.83436097 -0.84125573 ...  0.42224568  0.4081433
   0.39391627]
 [-0.35872031 -0.36181405 -0.36480441 ...  0.18304707  0.17693332
   0.17076544]] ... [[-0.46118508 -0.44608068 -0.43084806 ... -0.87534198 -0.88349422
  -0.89137455]
 [-0.82722792 -0.83436097 -0.84125573 ...  0.42224568  0.4081433
   0.39391627]
 [-0.35872031 -0.36181405 -0.36480441 ...  0.18304707  0.17693332
   0.17076544]]
CPU times: user 3min 30s, sys: 3.4 s, total: 3min 34s
Wall time: 3min 35s


In [5]:
observatoryXYZ.shape  , JDlist.shape 

((3, 20000), (20000,))

# --------- Convenience Functionality for Demos ---------------------
 - Convenience code to make database with required tables for the sake of subsequently demonstrating PreCalc / SQL

In [7]:
# --- Demonstrate some convenience code to make database with required tables ---
# -------------------------------------------------------------------------------

def convenience_func_create_db_and_tables():
    importlib.reload(sql)
    
    # In order to save data, we require sql-db to exist, so let's set that up...
    # Force deletion then creation of db...
    if os.path.isfile( sql.fetch_db_filepath() ):
        os.remove(sql.fetch_db_filepath())
    conn = sql.create_connection( sql.fetch_db_filepath() )
    cur  = conn.cursor()
    
    # Test creation of db
    assert os.path.isfile( os.path.join( sql.fetch_db_filepath() ) ), 'no db'

    # Create required table(s)
    sql.create_object_coefficients_table(conn)
    sql.create_objects_by_jdhp_table(conn)
    sql.create_object_desig_table(conn)
    

    # Double-check that this worked by getting the count of tables with the name
    # - if the count is 1, then table exists
    cur.execute('SELECT name from sqlite_master WHERE type = "table" AND name = "objects_by_jdhp"')
    assert len(cur.fetchone()) == 1 , 'jdhp table does not exist'
    cur.execute('SELECT name from sqlite_master WHERE type = "table" AND name = "object_coefficients"')
    assert len(cur.fetchone()) == 1 , 'coeff table does not exist'
    cur.execute('SELECT name from sqlite_master WHERE type = "table" AND name = "object_desig"')
    assert len(cur.fetchone()) == 1 , 'coeff table does not exist'

    return conn

# Create db & tables
conn = convenience_func_create_db_and_tables()
# Repeat check that db exists (other checks done within *convenience_func_create_db_and_tables()* )
os.path.isfile( os.path.join( sql.fetch_db_filepath() ) )

 *** WARNING TO MPC STAFF *** 
 THIS DEVELOPMENTAL CODE IS SAVING TO THE USERS-DIRECTORY 
 THIS SHOULD BE CHANGED TO A SINGLE LOCN ON MARSDEN 
 *** END OF WARNING ***
 *** WARNING TO MPC STAFF *** 
 THIS DEVELOPMENTAL CODE IS SAVING TO THE USERS-DIRECTORY 
 THIS SHOULD BE CHANGED TO A SINGLE LOCN ON MARSDEN 
 *** END OF WARNING ***
 *** WARNING TO MPC STAFF *** 
 THIS DEVELOPMENTAL CODE IS SAVING TO THE USERS-DIRECTORY 
 THIS SHOULD BE CHANGED TO A SINGLE LOCN ON MARSDEN 
 *** END OF WARNING ***
 *** WARNING TO MPC STAFF *** 
 THIS DEVELOPMENTAL CODE IS SAVING TO THE USERS-DIRECTORY 
 THIS SHOULD BE CHANGED TO A SINGLE LOCN ON MARSDEN 
 *** END OF WARNING ***
 *** WARNING TO MPC STAFF *** 
 THIS DEVELOPMENTAL CODE IS SAVING TO THE USERS-DIRECTORY 
 THIS SHOULD BE CHANGED TO A SINGLE LOCN ON MARSDEN 
 *** END OF WARNING ***


True

# --------- PreCalc ---------------------


 - A core part of the PreCalc routine is the generation of HP-positions on a nightly-basis
 - These calculations require that the positions of the observatory are provided as inputs: I.e. I want to completely separate the obs-posn stuff from the cheby evaluation calculations
 - For the sake of these demonstrations, I am going to generate a set of standardized input observatory-positions and then provide those as inputs. 
     

#### Tangential demo of using the *generate_HP()* function attached to the MSC class 

In [8]:
# (0) Create demo MSC(s) & create empty db
MSCs = create_sample_cheby_dict( n=4 )
conn = convenience_func_create_db_and_tables()

# How to calculate HP: 
# - Repeating the demo from the "Demonstrate_Orbital_Chebyshev_Functionality" notebook
# NB: need to restrict the queried dates to the those supported by the MSC
print( MSCs[0].get_valid_range_of_dates() ) 
indicees = np.where(JDlist < MSCs[0].get_valid_range_of_dates()[1] )[0]

HPs=MSCs[0].generate_HP( JDlist[indicees] , 
                observatoryXYZ[:,indicees], 
                APPROX = True )

# N.B. final shape = (3,len(times_TDB))
HPs.shape , JDlist[indicees].shape , observatoryXYZ[:,indicees].shape 


max(times):2459999.0
INIT MSC_Loader...
 *** WARNING TO MPC STAFF *** 
 THIS DEVELOPMENTAL CODE IS SAVING TO THE USERS-DIRECTORY 
 THIS SHOULD BE CHANGED TO A SINGLE LOCN ON MARSDEN 
 *** END OF WARNING ***
 *** WARNING TO MPC STAFF *** 
 THIS DEVELOPMENTAL CODE IS SAVING TO THE USERS-DIRECTORY 
 THIS SHOULD BE CHANGED TO A SINGLE LOCN ON MARSDEN 
 *** END OF WARNING ***
 *** WARNING TO MPC STAFF *** 
 THIS DEVELOPMENTAL CODE IS SAVING TO THE USERS-DIRECTORY 
 THIS SHOULD BE CHANGED TO A SINGLE LOCN ON MARSDEN 
 *** END OF WARNING ***
 *** WARNING TO MPC STAFF *** 
 THIS DEVELOPMENTAL CODE IS SAVING TO THE USERS-DIRECTORY 
 THIS SHOULD BE CHANGED TO A SINGLE LOCN ON MARSDEN 
 *** END OF WARNING ***
(2440000, 2459999.999999)


((20000,), (20000,), (3, 20000))

## Demonstrate the main PreCalc *upsert* func

#### This is high level code that hides a lot of details: to get a lower-level understanding of the SQL updates/queries, please look lower-down in this notebook at the detailed SQL section

In [9]:
importlib.reload(sql)
importlib.reload(precalc)

'''
Demonstrate the main PreCalc *upsert* func
- This     
    (i) uses the sql.insert_desig() function from above
    (ii) uses the sql.upsert_MSC() function from above
    (iii) uses the M.generate_HP() function from above
    (iv) uses the sql.insert_HP() function from above
    
'''
# (0) Create demo MSC(s) & create empty db
MSCs = create_sample_cheby_dict( n=4 )
conn = convenience_func_create_db_and_tables()

# Query the database to establish it exists, but that there's nothing in it
# - More detail on this can be found in the "low-level-sql" section of this notebook (lower-down)
cur = conn.cursor()
object_id = 1 
cur.execute("SELECT * FROM object_coefficients WHERE object_id=?", ( object_id, ))
print() 
print(f'Query results: EMPTY DB: len(cur.fetchall()={len(cur.fetchall())})') 
print() 

# Declare a "PreCalc" object 
P = precalc.PreCalc()

# Do the upsert 
P.upsert( MSCs , observatoryXYZ)

# Check the upsert worked by explicitly querying some quantities from the database ...
# - More detail on these queries be found in the "low-level-sql" section of this notebook (lower-down)
cur = conn.cursor()
cur.execute("SELECT * FROM object_desig WHERE object_id in (1,2,3,4)")
results = cur.fetchall()
print() 
print(f'Query results : POPULATED DB: len(results)={len(results)}') 
print() 
results

max(times):2459999.0
INIT MSC_Loader...
 *** WARNING TO MPC STAFF *** 
 THIS DEVELOPMENTAL CODE IS SAVING TO THE USERS-DIRECTORY 
 THIS SHOULD BE CHANGED TO A SINGLE LOCN ON MARSDEN 
 *** END OF WARNING ***
 *** WARNING TO MPC STAFF *** 
 THIS DEVELOPMENTAL CODE IS SAVING TO THE USERS-DIRECTORY 
 THIS SHOULD BE CHANGED TO A SINGLE LOCN ON MARSDEN 
 *** END OF WARNING ***
 *** WARNING TO MPC STAFF *** 
 THIS DEVELOPMENTAL CODE IS SAVING TO THE USERS-DIRECTORY 
 THIS SHOULD BE CHANGED TO A SINGLE LOCN ON MARSDEN 
 *** END OF WARNING ***
 *** WARNING TO MPC STAFF *** 
 THIS DEVELOPMENTAL CODE IS SAVING TO THE USERS-DIRECTORY 
 THIS SHOULD BE CHANGED TO A SINGLE LOCN ON MARSDEN 
 *** END OF WARNING ***

Query results: EMPTY DB: len(cur.fetchall()=0)

 *** WARNING TO MPC STAFF *** 
 THIS DEVELOPMENTAL CODE IS SAVING TO THE USERS-DIRECTORY 
 THIS SHOULD BE CHANGED TO A SINGLE LOCN ON MARSDEN 
 *** END OF WARNING ***

Query results : POPULATED DB: len(results)=4



[(1, '2022 AA'), (2, '2023 AA'), (3, '2024 AA'), (4, '2025 AA')]

## Demonstrate the main PreCalc *get_nightly_precalcs* func


In [11]:
importlib.reload(sql)
importlib.reload(precalc)
'''
 Demonstrate the main PreCalc *get_nightly_precalcs* func
 - This calls ...
'''
# Declare a "PreCalc" object 
P = precalc.PreCalc()

# We want to have some healpix that are in the database (so that the query can be successful)
# So let's use HP functionality that's part of the MSC class to get some appropriate HP for each date 
# - See "Demonstrate_Orbital_Chebyshev_Functionality" demo for more details 
n = 100
HPlist= MSCs[0].generate_HP( JDlist[:n] , 
                observatoryXYZ[:,:n], 
                APPROX = True )
print()
print(f'len(HPlist):{len(HPlist)}')
print(f' HPlist[:3]:{HPlist[:3]} ... HPlist[-3:]:{HPlist[-3:]}')
print()

# Now perform the main query against the precalculations for a given night & given set of healpix
print(f'Inputs to query ... JDlist[0], HPlist[:2] : {JDlist[0], HPlist[:2]}' )
MSCs = P.get_nightly_precalcs( JDlist[0], HPlist[:2])
for M in MSCs:
    print( f' type(M):{type(M)} , M.primary_unpacked_provisional_designation:{M.primary_unpacked_provisional_designation}' )


 *** WARNING TO MPC STAFF *** 
 THIS DEVELOPMENTAL CODE IS SAVING TO THE USERS-DIRECTORY 
 THIS SHOULD BE CHANGED TO A SINGLE LOCN ON MARSDEN 
 *** END OF WARNING ***


AttributeError: 'MSC' object has no attribute 'sector_init'

# --------- Lower-Level SQL Code ---------------------
 - Primarily used "behind the scenes"
 - Expect that the user will mainly be interacting with the above PreCalc functions

## Demonstrate code to "upsert" primary_unpacked_provisional_designation into checker sqlite db 
 - This will *NOT* be done in practice by the user, this will be done "behind-the-scenes" as part of another routine

In [12]:
importlib.reload(sql)

# (0) Create demo MSC(s) & create empty db
MSCs = create_sample_cheby_dict()

# (1) Low level direct call using sql
result = sql.insert_desig(conn ,MSCs[0].primary_unpacked_provisional_designation )
print('result = object_id = ' , result )

# (2) Query functionality ...
result = sql.query_number_by_desig(conn, MSCs[0].primary_unpacked_provisional_designation)
print('result = object_id = ' , result )


max(times):2459999.0
INIT MSC_Loader...
result = object_id =  1
result = object_id =  1


# Demonstrate code to "upsert" MSC into checker sqlite db 

In [13]:
# --- Demonstrate code to "upsert" MSC into mpchecker2 sqlite db ----------------
# - N.B. in practice a "higher level" function precalc.upsert() will be used ----
#        See cells below for example of precalc.upsert() function ...
# -------------------------------------------------------------------------------
importlib.reload(sql)

# (0) Create demo MSC(s) & create empty db
MSCs = create_sample_cheby_dict()
conn = convenience_func_create_db_and_tables()

# (1) Direct call using sql
object_id = result
sql.upsert_MSC(conn ,MSCs[0] , object_id )

# (2) Demonstrate that there is something in the database by querying it ... 
cur = conn.cursor()
cur.execute("SELECT * FROM object_coefficients WHERE object_id=?", ( object_id, ))

print('Query results ... : object_id ') 
for row in cur.fetchall(): 
    print(row[1] )#, pickle.loads( row[-1] ).shape  ) 


names = [description[0] for description in cur.description]
print('There are a lot of columns in the *object_coefficients* table, one for each sector ...')
print('column names = ' , names )


max(times):2459999.0
INIT MSC_Loader...
 *** WARNING TO MPC STAFF *** 
 THIS DEVELOPMENTAL CODE IS SAVING TO THE USERS-DIRECTORY 
 THIS SHOULD BE CHANGED TO A SINGLE LOCN ON MARSDEN 
 *** END OF WARNING ***
 *** WARNING TO MPC STAFF *** 
 THIS DEVELOPMENTAL CODE IS SAVING TO THE USERS-DIRECTORY 
 THIS SHOULD BE CHANGED TO A SINGLE LOCN ON MARSDEN 
 *** END OF WARNING ***
 *** WARNING TO MPC STAFF *** 
 THIS DEVELOPMENTAL CODE IS SAVING TO THE USERS-DIRECTORY 
 THIS SHOULD BE CHANGED TO A SINGLE LOCN ON MARSDEN 
 *** END OF WARNING ***
 *** WARNING TO MPC STAFF *** 
 THIS DEVELOPMENTAL CODE IS SAVING TO THE USERS-DIRECTORY 
 THIS SHOULD BE CHANGED TO A SINGLE LOCN ON MARSDEN 
 *** END OF WARNING ***
Query results ... : object_id 
1
There are a lot of columns in the *object_coefficients* table, one for each sector ...
column names =  ['coeff_id', 'object_id', 'sector_0_2440000', 'sector_1_2440032', 'sector_2_2440064', 'sector_3_2440096', 'sector_4_2440128', 'sector_5_2440160', 'sector_6_

# --- Demonstrate code to query checker db for object-coefficients
 - Again, this would likely *NOT* be performed by the user directly (but would instead be called by some other function, and/or not be used at all, except during development) 

In [14]:
# Set up connection to db to enable subseqjuent low-level query  
importlib.reload(sql)
conn = sql.create_connection( sql.fetch_db_filepath() )

# Execute the query (searching against primary_unpacked_provisional_designation == '2022 AA' )
print('Input desig ... ', object_id )
result = sql.query_object_coefficients(conn , object_id)

# Examine the characteristics of the output 
print('Characteristics of query results ...' , type(result) , ' dict-length=', len(result) )
key0 = list(result.keys())[0] ; print('\t'*2, ' key0 ... ', key0, result[key0].shape )

# Compare the output to the input to verify is the same ... 
if np.all(MSCs[0].sector_coeffs[ int(key0.split("_")[1]) ] == result[key0] ) : print( 'In == Out !!')

 *** WARNING TO MPC STAFF *** 
 THIS DEVELOPMENTAL CODE IS SAVING TO THE USERS-DIRECTORY 
 THIS SHOULD BE CHANGED TO A SINGLE LOCN ON MARSDEN 
 *** END OF WARNING ***
Input desig ...  1
Characteristics of query results ... <class 'dict'>  dict-length= 625
		  key0 ...  sector_0_2440000 (8, 27)
In == Out !!


### --- query checker db for object-coefficients over a limited subset of sectors
 - Again, this would likely *NOT* be performed by the user directly (but would instead be called by some other function) 

In [164]:
# Set up connection to db to enable subseqjuent low-level query  
importlib.reload(sql)
conn = sql.create_connection( sql.fetch_db_filepath() )

# Execute the query (searching against primary_unpacked_provisional_designation == '2022 AA' )
print('Input desig ... ', object_id )
result = sql.query_object_coefficients(conn , 
                                        object_id , 
                                        sector_numbers = range(0,4))
# Examine the characteristics of the output 
print('Characteristics of query results ...' , type(result) , len(result) )
for key in result.keys() : 
    print('\t'*2, key, ' : ', result[key].shape )

# Compare the output to the input to verify is the same ... 
if np.all(MSCs[0].sector_coeffs[ int(key0.split("_")[1]) ] == result[key0] ) : print( 'In == Out !!')

 *** WARNING TO MPC STAFF *** 
 THIS DEVELOPMENTAL CODE IS SAVING TO THE USERS-DIRECTORY 
 THIS SHOULD BE CHANGED TO A SINGLE LOCN ON MARSDEN 
 *** END OF WARNING ***
Input desig ...  1
Characteristics of query results ... <class 'dict'> 4
		 sector_0_2440000  :  (18, 27)
		 sector_1_2440032  :  (18, 27)
		 sector_2_2440064  :  (18, 27)
		 sector_3_2440096  :  (18, 27)
In == Out !!


#### Generate a set of Healpix based on supplied observatory positions (center of the Earth) from the top of this notebook 

In [165]:
%%time
# Now explicitly evaluate the HP functionality to get the appropriate HP for each date 
# - See "Demonstrate_Orbital_Chebyshev_Functionality" demo for more details 
n = 15000
HPlist= MSCs[0].generate_HP( JDlist[:n] , 
                observatoryXYZ[:,:n], 
                APPROX = True )
print(f'len(HPlist):{len(HPlist)}')
print(f' HPlist[:3]:{HPlist[:3]} ... HPlist[-3:]:{HPlist[-3:]}')

len(HPlist):15000
 HPlist[:3]:[1393 1393 1396] ... HPlist[-3:]:[1718 1718 1716]
CPU times: user 85.5 ms, sys: 10.8 ms, total: 96.3 ms
Wall time: 94.6 ms


### Here is where we do the actual upsert into the db

In [166]:
importlib.reload(sql)
# Now the code to do the insert ...
sql.insert_HP(conn, JDlist[:n], HPlist, object_id)


### Demonstrate code to query Healpix by desig ...

In [168]:
# sql query ...
importlib.reload(sql)
result = sql.query_jd_hp(conn, JDlist[0], HPlist[:2])
print(JDlist[0], HPlist[:2])
# Examine the characteristics of the output 
print('Characteristics of query results ...' , type(result) , len(result) )
print(result)

# Compare the output to the input to verify is the same ... 
#if np.all(MSCs[0].sector_coeffs[ int(key0.split("_")[1]) ] == result[key0] ) : print( 'In == Out !!')

2440000 [1393 1393]
Characteristics of query results ... <class 'list'> 1
[1]


# Joining together some of the above:
 - ## Querying objects & coefficient-sets by JD & HP-list 

In [285]:
%%time 
importlib.reload(sql)

# Execute the query (searching against JDlist[0], HPlist[:2] = 2440000 [1393 1393] )
print('Input = ... ', JDlist[0], HPlist[:2] )
result_dict = sql.query_coefficients_by_jd_hp(conn, JDlist[0], HPlist[:2])
for key in result_dict.keys():
    print(key, len(result_dict[key]))

# Examine the characteristics of the output 
#print('Characteristics of query results ...' , type(result) , ' dict-length=', len(result) )
#key0 = list(result.keys())[0] ; print('\t'*2, ' key0 ... ', key0, result[key0].shape )



Input = ...  2440000 [1393 1393]
1 624
2 624
3 624
4 624
CPU times: user 39.9 ms, sys: 8.57 ms, total: 48.5 ms
Wall time: 46.4 ms


In [291]:
importlib.reload(sql)
result = sql.query_desig_by_number(conn, [1,2,3])
result

[(1, '2022 AA'), (2, '2023 AA'), (3, '2024 AA')]